# App Rating
* Important note:
* Any transformation we do should be contained in a function so we can reuse it later

## Download data from kaggle

In [ ]:
import pandas as pd

train_df = pd.read_csv("app-rating-competition/train.csv")
test_df = pd.read_csv("app-rating-competition/test.csv")
sample_submission_df = pd.read_csv("app-rating-competition/SampleSubmission.csv")

## Data Exploration

In [ ]:
train_df.head()

In [ ]:
train_df

### Column Observation
* X0 → App Name (Names)
    * Has duplicates
* X1 → App Category (Categorical)
    * Most frequent is FAMILY (1605)
    * 34 distinct categories
* X2 → Size in bytes probably (Numerical)
    * Could be used for weighted rating or normalization
* X3 → Downloads in millions (Numerical)
    * Has 1359 entries "Varies with device"
    * We might have to edit this with the app name duplicates
* X4 → Number of reviews (Categorical)
* X5 → Free/Paid (Categorical)
    * Has one null we should explore before handling
* X6 → Price if paid (Numerical)
    * Check for any free with price or any paid with 0
* X7 → Age Rating (Categorical)
    * 6 categories
    * Has a null value we should check before handling
    * Top is Everyone (7222)
* X8 → App Tags (Categorical) (Should be enhanced)
    * Most frequent is Tools (705)
* X9 → Release Date (Will be heavily feature extracted)
    * Month/Day/Year
    * Maybe convert to days (released on weekend etc.)
    * Could find if released on specific holidays (Christmas, etc.)
* X10 → App Version (I think) (Numerical or Categorical)
    * 7 null values
    * Top is "Varies with device" 1173
* X11 → Compatible OS version (I think) (Categorical)
    * 3 Null values
    * Top is "4.1 and up" (2072)
* Y → App Rating
    * 1474 Null values (should be cleaned)
    * Has outliers heavily

### Rename Columns

In [ ]:
def rename_columns(dataframe):
    reformated_df = dataframe.rename(
        columns={"X0": "app_name", "X1": "app_category", "X2": "size_unknown_unit", "X3": "downloads_unstandardized",
                 "X4": "reviews_count", "X5": "free_paid", "X6": "price_if_paid", "X7": "age_rating",
                 "X8": "app_tags", "X9": "release_date", "X10": "app_version",
                 "X11": "compatible_os_version"})
    return reformated_df

In [ ]:
train_df1 = rename_columns(train_df)
train_df1

In [ ]:
train_df_copy = train_df1.copy()

### Explore Null Values

#### Free/Paid nulls

In [ ]:
train_df_copy[train_df_copy.free_paid.isnull()]

* This has size 0, non-specific downloads, zero reviews, and null Y, so it sounds safe to drop
* Out of curiosity let's check if any other apps exist with the same name

In [ ]:
train_df_copy[train_df_copy['app_name'].str.contains("conquer", case=False, na=False)]

* So we will drop this

In [ ]:
train_df_copy.dropna(subset=['free_paid'], inplace=True)  # drops column 7533

#### Age Rating Nulls

In [ ]:
train_df_copy[train_df_copy.age_rating.isnull()]

* It looks like this is perfectly ok, but the values of columns are switched around
* We can check for all distinct values from now to find any similar cases

In [ ]:
train_df_copy.loc[8653]

##### Real Values
* app_category: compatible_os_version
* size_bytes: Y
* downloads_millions: size_bytes
* reviews_count: downloads_millions
* free_paid: reviews_count
* price_if_paid: free_paid
* age_rating: price_if_paid
* app_tags: age_rating
* release_date: app_tags
* app_version: release_date
* compatible_os_version: app_version
* Y: app_category

In [ ]:
swap_pairs = [
    ("app_category", "compatible_os_version"),
    ("size_bytes", "Y"),
    ("downloads_millions", "size_bytes"),
    ("reviews_count", "downloads_millions"),
    ("free_paid", "reviews_count"),
    ("price_if_paid", "free_paid"),
    ("age_rating", "price_if_paid"),
    ("app_tags", "age_rating"),
    ("release_date", "app_tags"),
    ("app_version", "release_date"),
    ("compatible_os_version", "app_version"),
    ("Y", "app_category"),
]

# First read all col2 values
values_to_copy = {
    col1: train_df_copy.at[8653, col2]
    for col1, col2 in swap_pairs
}

# Then write them all at once
for col1, value in values_to_copy.items():
    train_df_copy.at[8653, col1] = value



In [ ]:
train_df_copy.loc[8653]

* The row is still not fully clean
* Also I'm not sure 19.0 is possible in size as other sizes aren't in float
* And not sure rating is real
* Category and tags are null but they can be easily found online
* We will drop this for now but save it

In [ ]:
dropped_row_8653 = train_df_copy.loc[[8653]]

train_df_copy.drop(8653, inplace=True)
dropped_row_8653

#### App Version Nulls

In [ ]:
train_df_copy[train_df_copy.app_version.isnull()]

* Turns out downloads are not only in millions, so let's edit the name, and we will solve this later
* I don't know what the size is in; it wouldn't make sense for it to be bytes, so let's check later
* Also notice the last four rows have null ratings as well
* But the rest have valid columns, so we could drop the last 4 for now and come up with app_version values for the rest

In [ ]:
dropped_rows_null_version_rating = train_df_copy[
    train_df_copy[['app_version', 'Y']].isnull().all(axis=1)
]
dropped_rows_null_version_rating

In [ ]:
train_df_copy.drop(dropped_rows_null_version_rating.index, inplace=True)
train_df_copy[train_df_copy.app_version.isnull()]

* Now rename columns as stated

In [ ]:
train_df_copy.rename(columns={
    'size_bytes': 'size_unknown_unit',
    'downloads_millions': 'downloads_unstandardized'
}, inplace=True)
train_df_copy.head()


#### Compatible OS Nulls

In [ ]:
train_df_copy[train_df_copy.compatible_os_version.isnull()]

* substratum is a theme engine for android to allow users to set themes to their phone
* Pi Dark is 2MB in the play store, so the unit we have still doesn't make sense
* I'm not sure if we should drop these, so we'll leave them for now
* Let's look at any other substratum

In [ ]:
train_df_copy[train_df_copy['app_name'].str.contains("substratum", case=False, na=False)]

* All substratum themes have almost the same compatible_os_version as either 6.0 and up or 7.0 and up
* So we can fill our null values using this
* I will set both as 7.0 and up since it is most common

In [ ]:
train_df_copy.fillna(value={"compatible_os_version": "7.0 and up"}, inplace=True)
train_df_copy[train_df_copy['app_name'].str.contains("substratum", case=False, na=False)]

#### App Rating nulls

In [ ]:
train_df_copy[train_df_copy.Y.isnull()]

In [ ]:
train_df_copy.Y.isnull().sum()

* We have 1469 nulls which is too much to handle, so we can return to this later after more cleaning

### Explore Data Types
* Data types for a column should be consistent, so we should look for inconsistent rows and clean them

In [ ]:
train_df_copy.dtypes

* Let's check for individual columns types as well as all unique values where possible

#### App Name

In [ ]:
train_df_copy['app_name'].map(type).value_counts()


In [ ]:
train_df_copy['app_name'].value_counts()

* Duplicated Names could cause a problem, we'll check later

#### App Category

In [ ]:
train_df_copy['app_category'].map(type).value_counts()

In [ ]:
train_df_copy['app_category'].value_counts()

#### Size

In [ ]:
train_df_copy['size_unknown_unit'].map(type).value_counts()

In [ ]:
train_df_copy['size_unknown_unit'].value_counts()

* Apps with size 0 could cause a problem, so check later
* Also, size shouldn't be string unless we turn it categorical (<100, <10, etc.)

#### Downloads

In [ ]:
train_df_copy['downloads_unstandardized'].map(type).value_counts()

In [ ]:
train_df_copy['downloads_unstandardized'].value_counts()

* Varies with device could cause a problem, but we should turn this numerical.
* We only have numbers in millions (M) and thousands (K), so we should standardize

#### Reviews Number

In [ ]:
train_df_copy['reviews_count'].map(type).value_counts()

In [ ]:
train_df_copy['reviews_count'].value_counts()

* Looks fine but we should try it as categorical and numerical

#### Free/Paid

In [ ]:
train_df_copy['free_paid'].map(type).value_counts()

In [ ]:
train_df_copy['free_paid'].value_counts()

* Looks fine and will be categorical
* Let's look at any incorrect mappings (free but has price, paid but price 0

#### Price if paid

In [ ]:
train_df_copy['price_if_paid'].map(type).value_counts()

In [ ]:
train_df_copy['price_if_paid'].value_counts()

In [ ]:
inconsistent_price_rows = train_df_copy[
    ((train_df_copy['free_paid'] == "Free") & (train_df_copy['price_if_paid'] != "0")) |
    ((train_df_copy['free_paid'] == "Paid") & (
            (train_df_copy['price_if_paid'] == "0") | (train_df_copy['price_if_paid'].isnull())))
    ]
inconsistent_price_rows

* Looks fine, we should remove the dollar sign to make it numerical
* Maybe try price tiers categories
* We also have no inconsistent prices

#### Age Rating

In [ ]:
train_df_copy['age_rating'].map(type).value_counts()

In [ ]:
train_df_copy['age_rating'].value_counts()

* Looks fine maybe clean up the string

#### App Tags

In [ ]:
train_df_copy['app_tags'].map(type).value_counts()

In [ ]:
train_df_copy['app_tags'].value_counts()

* This is currently too much and will need cleaning

#### Release Date

In [ ]:
train_df_copy['release_date'].map(type).value_counts()

In [ ]:
train_df_copy['release_date'].value_counts()

In [ ]:
expected_format = "%B %d, %Y"

invalid_dates = pd.to_datetime(
    train_df_copy['release_date'],
    format=expected_format,
    errors='coerce'  # Converts invalid formats to NaT (null)
).isna()

invalid_date_rows = train_df_copy[invalid_dates]
invalid_date_rows

* All date rows are consistent in format
* However, this current release date will not help and will need to be extracted to more useful features

#### App Version

In [ ]:
train_df_copy['app_version'].map(type).value_counts()

In [ ]:
train_df_copy['app_version'].value_counts()

* Again varies with device is causing a problem here
* And we want to standardize the type here

In [ ]:
train_df_copy[train_df_copy['app_version'].map(type) == float]

In [ ]:
dropped_rows_null_version = train_df_copy[
    train_df_copy[['app_version']].isnull().all(axis=1)
]
dropped_rows_null_version

In [ ]:
train_df_copy.dropna(subset=['app_version'], inplace=True)

* We'll drop these rows for now

#### Compatible OS

In [ ]:
train_df_copy['compatible_os_version'].map(type).value_counts()

In [ ]:
train_df_copy['compatible_os_version'].value_counts()

* Again varies with device
* Also some fields have "5.0 and up" format and others have "5.0 - 7.1.1" format, we should explore these formats

#### App Rating

In [ ]:
train_df_copy['Y'].map(type).value_counts()

In [ ]:
train_df_copy['Y'].value_counts()

### Handling Duplicates

#### Duplicate Names

In [ ]:
train_df_copy['app_name'].value_counts()

In [ ]:
roblox_rows = train_df_copy[train_df_copy['app_name'] == "ROBLOX"]
roblox_rows

* These are duplicates, we will turn them to 1 and use the GAME tag as its the median
* We will get the mean size as it's different

In [ ]:
app_category_median = roblox_rows['app_category'].mode()[0]

app_category_median

In [ ]:
size_mean = pd.to_numeric(roblox_rows['size_unknown_unit'], errors='coerce').mean()

size_mean

In [ ]:
def drop_duplicates_specific_value(dataframe, column_name, value_name):
    is_column = dataframe[column_name] == value_name
    column_deduped = dataframe[is_column].drop_duplicates(subset=column_name, keep='first')

    non_column = dataframe[~is_column]

    return pd.concat([non_column, column_deduped], ignore_index=True)


In [ ]:
train_df_copy = drop_duplicates_specific_value(train_df_copy, 'app_name', 'ROBLOX')
train_df_copy[train_df_copy['app_name'] == "ROBLOX"]

In [ ]:
import math

roblox_size_floored_str = str(math.floor(size_mean))
roblox_size_floored_str

In [ ]:
train_df_copy.loc[train_df_copy['app_name'] == "ROBLOX", 'size_unknown_unit'] = roblox_size_floored_str
train_df_copy[train_df_copy['app_name'] == "ROBLOX"]

* Let's now handle other duplicates

In [ ]:
train_df_copy['app_name'].value_counts()

#### Drop perfect duplicates
* Let's drop perfect duplicates first (all columns equal)

In [ ]:
train_df_copy_checkpoint = train_df_copy.copy()

In [ ]:
train_df_copy.drop_duplicates(inplace=True)
train_df_copy['app_name'].value_counts()

#### Duplicates w/ different sizes
* We still have duplicates, so we will handle them in different ways
* Let's first check for duplicates with different sizes only

In [ ]:
def find_near_duplicates(df, except_col):
    columns_except = df.columns.difference([except_col])
    near_duplicates = df[df.duplicated(subset=columns_except, keep=False)]
    return near_duplicates


# Define columns to check for duplicates (exclude 'size_unknown_unit')
columns_except_size = train_df_copy.columns.difference(['size_unknown_unit'])

# Find duplicates based on all columns except size
near_duplicates = find_near_duplicates(train_df_copy, except_col='size_unknown_unit')
near_duplicates

* We will now take the mean size and drop duplicates and use the floored mean
* This is a helper function that we can use later

In [ ]:
def deduplicate_with_aggregation(dataframe, group_by_except_col, target_col, strategy, key_col):
    """
    Deduplicates rows that are equal in all columns except one,
    aggregates the target column using a given strategy, and
    keeps only one row per duplicate group with the updated value.

    Parameters:
    - dataframe: pd.DataFrame
    - group_by_except_col: str → the column to ignore when checking for duplicates
    - target_col: str → the column to aggregate and update (usually same as group_by_except_col)
    - strategy: str → 'mean', 'median', or 'mode'
    - key_col: str → the column to identify duplicate groups (default 'app_name')

    Returns:
    - Cleaned DataFrame with duplicates dropped and target_col updated.
    """

    group_cols = dataframe.columns.difference([group_by_except_col])
    duplicate_rows = dataframe[dataframe.duplicated(subset=group_cols, keep=False)]
    grouped = duplicate_rows.groupby(list(group_cols))

    for _, group_df in grouped:
        # Aggregate value based on strategy
        values = group_df[target_col].dropna()

        if strategy == 'mean':
            result = pd.to_numeric(values, errors='coerce').mean()
            aggregated_value = str(math.floor(result)) if pd.notna(result) else None

        elif strategy == 'median':
            result = pd.to_numeric(values, errors='coerce').median()
            aggregated_value = str(math.floor(result)) if pd.notna(result) else None

        elif strategy == 'mode':
            mode_vals = values.mode()
            aggregated_value = str(mode_vals[0]) if not mode_vals.empty else None

        else:
            raise ValueError(f"Unsupported strategy: {strategy}")

        # Representative key value for filtering
        key_value = group_df[key_col].iloc[0]

        # Drop duplicates of this key
        dataframe = drop_duplicates_specific_value(dataframe, key_col, key_value)

        # Assign new value if available
        if aggregated_value is not None:
            dataframe.loc[dataframe[key_col] == key_value, target_col] = aggregated_value

    return dataframe


In [ ]:
train_df_copy = deduplicate_with_aggregation(
    train_df_copy,
    group_by_except_col='size_unknown_unit',
    target_col='size_unknown_unit',
    strategy='mean',
    key_col='app_name'
)

find_near_duplicates(train_df_copy, except_col='size_unknown_unit')

#### Duplicates w/ different categories

* Let's first check if this has helped any of our other cases

In [ ]:
train_df_copy.Y.isnull().sum()

In [ ]:
(train_df_copy['compatible_os_version'] == "Varies with device").sum()

In [ ]:
(train_df_copy['app_version'] == "Varies with device").sum()

In [ ]:
(train_df_copy['downloads_unstandardized'] == "Varies with device").sum()

* Null Y: 1469 -> 1460
* OS varies: 1087 -> 859
* App version varies: 1172 -> 920
* downloads varies: 1358 -> 1062
* Let's now find duplicates with different category

In [ ]:
find_near_duplicates(train_df_copy, except_col='app_category')

* Using a median wouldn't apply here as we only have two values for each
* However, looking at the tags and name we can decide which category is most suitable
* Learn C++: EDUCATION
* Fuzzy Numbers: EDUCATION
* Candy Bomb: GAME

In [ ]:
train_df_copy.loc[train_df_copy['app_name'] == "Learn C++", 'app_category'] = "EDUCATION"
train_df_copy.loc[train_df_copy['app_name'] == "Fuzzy Numbers: Pre-K Number Foundation", 'app_category'] = "EDUCATION"
train_df_copy.loc[train_df_copy['app_name'] == "Candy Bomb", 'app_category'] = "GAME"
train_df_copy.drop_duplicates(inplace=True)
find_near_duplicates(train_df_copy, except_col='app_category')

#### Duplicates w/ different release date
* Only other near duplicates are release date

In [ ]:
find_near_duplicates(train_df_copy, except_col='release_date')

* Again we will set this manually
* osmino Wi-Fi: free WiFi: August 6, 2018
* Target - now with Cartwheel:  July 25, 2018

In [ ]:
train_df_copy.loc[train_df_copy['app_name'] == "Target - now with Cartwheel", 'release_date'] = "July 25, 2018"
train_df_copy.loc[train_df_copy['app_name'] == "osmino Wi-Fi: free WiFi", 'release_date'] = "August 6, 2018"
train_df_copy.drop_duplicates(inplace=True)
find_near_duplicates(train_df_copy, except_col='release_date')

#### Duplicate Names Check

In [ ]:
train_df_copy['app_name'].value_counts()

In [ ]:
app_names_with_3 = train_df_copy['app_name'].value_counts()
app_names_with_3 = app_names_with_3[app_names_with_3 == 3].index
rows_with_3_duplicates = train_df_copy[train_df_copy['app_name'].isin(app_names_with_3)]
rows_with_3_duplicates

* Most of these make sense other than the video editor which has 2 duplicates from the 3

In [ ]:
train_df_copy.loc[
    (train_df_copy['app_name'] == "Video Editor") &
    (train_df_copy['reviews_count'] == "5,000,000+"),
    'app_category'
] = "VIDEO_PLAYERS"

rows_with_3_duplicates = train_df_copy[train_df_copy['app_name'].isin(app_names_with_3)]
rows_with_3_duplicates

In [ ]:
train_df_copy.loc[
    (train_df_copy['app_name'] == "Video Editor") &
    (train_df_copy['reviews_count'] == "5,000,000+"),
    'size_unknown_unit'
] = "159620"

In [ ]:
train_df_copy.drop_duplicates(inplace=True)
rows_with_3_duplicates = train_df_copy[train_df_copy['app_name'].isin(app_names_with_3)]
rows_with_3_duplicates

* Duplicated names with 2 are too much so we will have to use other ways

#### Duplicates w/ different sizes & categories

In [ ]:
def find_almost_near_duplicates(df, except_cols):
    """
    Finds near-duplicate rows in a DataFrame, ignoring specified columns.

    Parameters:
    - df: pd.DataFrame
    - except_cols: list of str → columns to exclude when checking for duplicates

    Returns:
    - DataFrame containing near-duplicate rows
    """
    columns_except = df.columns.difference(except_cols)
    near_duplicates = df[df.duplicated(subset=columns_except, keep=False)]
    return near_duplicates


In [ ]:
near_duplicates = find_almost_near_duplicates(train_df_copy, except_cols=['size_unknown_unit', 'app_category'])
near_duplicates

* We have 64 of them, however we can notice that some fields have no download data
* We can drop these for now and think how we can use them later

##### Drop rows with all varies

In [ ]:
dropped_rows_varies_downloads_version_os = near_duplicates[
    (near_duplicates['downloads_unstandardized'] == "Varies with device") &
    (near_duplicates['app_version'] == "Varies with device") &
    (near_duplicates['compatible_os_version'] == "Varies with device")
    ]

In [ ]:
train_df_copy = train_df_copy.drop(index=dropped_rows_varies_downloads_version_os.index)

In [ ]:
near_duplicates = find_almost_near_duplicates(train_df_copy, except_cols=['size_unknown_unit', 'app_category'])
near_duplicates

In [ ]:
def deduplicate_with_mean_and_tag_based_category(df, key_col='app_name'):
    """
    Deduplicates rows that are identical in all columns except for
    'size_unknown_unit' and 'app_category'. Uses:

    - Floored mean for size
    - Original logic: if exactly one category is found in app_tags → keep that
    - If not, use category_tag_map to infer from tags
    - If still ambiguous or no match, keep all rows

    Parameters:
    - df: DataFrame to clean
    - key_col: Column to identify unique groups (e.g., 'app_name')

    Returns:
    - Cleaned DataFrame with resolved duplicates
    """

    # Category-to-tag mapping
    category_tag_map = {
        "ART_AND_DESIGN": ["art & design"],
        "GAME": ["simulation", "action", "role playing", "puzzle"],
        "FAMILY": ["casual"]
    }

    except_cols = ['size_unknown_unit', 'app_category']
    group_cols = df.columns.difference(except_cols)
    duplicate_rows = df[df.duplicated(subset=group_cols, keep=False)]
    grouped = duplicate_rows.groupby(list(group_cols))

    resolved_rows = []
    skipped_indexes = []

    for _, group_df in grouped:
        size_values = pd.to_numeric(group_df['size_unknown_unit'], errors='coerce')
        size_mean = str(math.floor(size_values.mean())) if not size_values.isna().all() else None

        category_values = group_df['app_category'].dropna().unique()
        app_tags = str(group_df['app_tags'].iloc[0]) if 'app_tags' in group_df else ""
        app_tags_lower = app_tags.lower()

        ### Step 1: Try original logic — exact category in tags
        matched_direct = [cat for cat in category_values if cat.lower().replace('_', ' ') in app_tags_lower]

        if len(matched_direct) == 1:
            chosen_category = matched_direct[0]
        else:
            ### Step 2: Try using custom tag map
            matched_from_map = []
            for cat in category_values:
                if cat in category_tag_map:
                    for keyword in category_tag_map[cat]:
                        if keyword.lower() in app_tags_lower:
                            matched_from_map.append(cat)
                            break
            matched_from_map = list(set(matched_from_map))

            chosen_category = matched_from_map[0] if len(matched_from_map) == 1 else None

        if chosen_category:
            resolved_row = group_df.iloc[0].copy()
            resolved_row['size_unknown_unit'] = size_mean
            resolved_row['app_category'] = chosen_category
            resolved_rows.append(resolved_row)
        else:
            skipped_indexes.extend(group_df.index.tolist())

    # Drop only those that were resolved
    cleaned_df = df.drop(index=duplicate_rows.index.difference(skipped_indexes))
    cleaned_df = pd.concat([cleaned_df, pd.DataFrame(resolved_rows)], ignore_index=True)

    return cleaned_df


* This function will take the mean for the size
* For the category it will look if one of the categories match the tags, if so it will use that; if neither or both match it will not drop the column
* We also add custom mappings for vague tags

In [ ]:
train_df_copy = deduplicate_with_mean_and_tag_based_category(train_df_copy)
near_duplicates = find_almost_near_duplicates(train_df_copy, except_cols=['size_unknown_unit', 'app_category'])
near_duplicates

In [ ]:
train_df_copy.to_csv("cleaned_train_df.csv", index=False)

### Handle Varies with device

In [ ]:
import pandas as pd

train_df_copy = pd.read_csv("submissions/cleaned_train_df.csv")

In [ ]:
train_df_copy.Y.isnull().sum()

In [ ]:
(train_df_copy['compatible_os_version'] == "Varies with device").sum()

In [ ]:
(train_df_copy['app_version'] == "Varies with device").sum()

In [ ]:
(train_df_copy['downloads_unstandardized'] == "Varies with device").sum()

* Null Y: 1469 -> 1458
* OS varies: 1087 -> 845
* App version varies: 1172 -> 906
* downloads varies: 1358 -> 1047

* For now we can remove any varies with device in download and with a null Y
* Also remove any that has varies in all 3

In [ ]:
dropped_rows_varies_downloads_null_y = train_df_copy[
    (train_df_copy['downloads_unstandardized'] == "Varies with device") &
    (train_df_copy['Y'].isnull())
    ]
dropped_rows_varies_downloads_null_y

In [ ]:
train_df_copy = train_df_copy.drop(index=dropped_rows_varies_downloads_null_y.index)

In [ ]:
dropped_rows_varies_all = train_df_copy[
    (train_df_copy['downloads_unstandardized'] == "Varies with device") &
    (train_df_copy['app_version'] == "Varies with device") &
    (train_df_copy['compatible_os_version'] == "Varies with device")
    ]
dropped_rows_varies_all

In [ ]:
train_df_copy = train_df_copy.drop(index=dropped_rows_varies_all.index)

In [ ]:
train_df_copy.Y.isnull().sum()

In [ ]:
train_df_copy.to_csv("cleaned_train_df2.csv", index=False)

In [ ]:
import pandas as pd

train_df_copy = pd.read_csv("submissions/cleaned_train_df2.csv")

### Final Cleanup for now

In [ ]:
train_df_copy.dropna(subset=['Y'], inplace=True)

In [ ]:
train_df_copy.to_csv("cleaned_train_only_dropped_y.csv", index=False)

In [ ]:
train_df_copy

In [ ]:
dropped_rows_varies_downloads = train_df_copy[train_df_copy['downloads_unstandardized'] == "Varies with device"]
dropped_rows_varies_downloads

In [ ]:
train_df_copy.drop(index=dropped_rows_varies_downloads.index, inplace=True)
train_df_copy

In [ ]:
dropped_rows_varies_os = train_df_copy[train_df_copy['compatible_os_version'] == "Varies with device"]
train_df_copy.drop(index=dropped_rows_varies_os.index, inplace=True)

# Leave this for now as we will drop the column in this iteration
# dropped_rows_varies_version = train_df_copy[train_df_copy['app_version'] == "Varies with device"]
# train_df_copy.drop(index=dropped_rows_varies_version.index, inplace=True)

train_df_copy


#### Cleanup columns
* We will convert downloads to a numeric column
* Convert reviews count to numeric and categorical columns

In [ ]:
train_df_copy['downloads_unstandardized'].value_counts()

In [ ]:
def parse_number(value):
    """
    Convert strings like '$1.5M', '$600K', '$100,000+' into integers.
    Strips '$', ',', and '+' before processing.
    """
    if isinstance(value, str):
        value = value.strip().upper().replace("$", "").replace(",", "").replace("+", "")
        try:
            if value.endswith("M"):
                return int(float(value[:-1]) * 1_000_000)
            elif value.endswith("K"):
                return int(float(value[:-1]) * 1_000)
            elif value.replace('.', '', 1).isdigit():
                return int(float(value))
        except ValueError:
            return None
    elif isinstance(value, (int, float)):
        return int(value)
    return None


In [ ]:
train_df_copy['downloads_unstandardized'] = train_df_copy['downloads_unstandardized'].apply(parse_number)
train_df_copy.rename(columns={
    'downloads_unstandardized': 'downloads'
}, inplace=True)
train_df_copy

In [ ]:
train_df_copy['reviews_count_numerical'] = train_df_copy['reviews_count'].apply(parse_number)
train_df_copy.rename(columns={
    'reviews_count': 'reviews_count_categorical'
})

In [ ]:
train_df_copy['price_if_paid'] = train_df_copy['price_if_paid'].apply(parse_number)
train_df_copy

* Note that if sorted by descending price we have many apps for "I am rich"

#### Dropping app tags
* We will drop the app tags for now, but we will return to them later and turn them into categories using dimensionality reduction

In [ ]:
train_df_copy.drop(columns=['app_tags'], inplace=True)

#### Handling Release date
* We will extract a couple of features from the release date
* Such as year, month, isWeekend, isHoliday, etc.

In [ ]:
train_df_copy['release_date'] = pd.to_datetime(train_df_copy['release_date'])

train_df_copy['year'] = train_df_copy['release_date'].dt.year
train_df_copy['month'] = train_df_copy['release_date'].dt.month
train_df_copy['day'] = train_df_copy['release_date'].dt.day
train_df_copy['weekday'] = train_df_copy['release_date'].dt.weekday
train_df_copy['quarter'] = train_df_copy['release_date'].dt.quarter
train_df_copy['is_weekend'] = train_df_copy['release_date'].dt.weekday >= 5
train_df_copy['is_month_start'] = train_df_copy['release_date'].dt.is_month_start
train_df_copy['is_month_end'] = train_df_copy['release_date'].dt.is_month_end
train_df_copy['week_of_year'] = train_df_copy['release_date'].dt.isocalendar().week
# Defined holiday season: November and December
train_df_copy['is_holiday_season'] = train_df_copy['release_date'].dt.month.isin([11, 12])
train_df_copy

In [ ]:
import holidays

us_holidays = holidays.US(years=range(2010, 2019))
for date, name in sorted(us_holidays.items()):
    print(f"{date}: {name}")


In [ ]:
train_df_copy['is_holiday'] = train_df_copy['release_date'].isin(us_holidays)
train_df_copy['holiday_name'] = train_df_copy['release_date'].map(us_holidays).fillna('Not Holiday')

train_df_copy

#### Handling App version & os version
* Let's first check how many of these fields are not in the expected format

In [ ]:
import re

invalid_versions = train_df_copy[~train_df_copy['app_version'].astype(str).str.match(r'^\d+(\.\d+)*$')]
invalid_versions

* 264 rows are in different format, so we will leave this for now
* However, this is a very important field as we can extract information from it such as
* if app is in bets, alpha, release
* How many iterations the app had
* How active is the app in development and updates

In [ ]:
# drop app versions for now
train_df_copy.drop(columns=['app_version'], inplace=True)

In [ ]:
invalid_os_versions = train_df_copy[~train_df_copy['compatible_os_version']
.astype(str)
.str.match(r'^\d+(\.\d+)* and up$')]
invalid_os_versions

* We can notice some versions have an upper limit for the version
* Some others have 4.4W which means android wear (watch)
* We can extract min os version, is android wear,

In [ ]:
def extract_min_base_os(value):
    value = str(value).upper().strip()

    # Match standard version patterns: '5.0 and up', '5.0', '5.0 - 6.0', etc.
    match = re.search(r'(\d+\.\d+)', value)
    if match:
        return match.group(1)

    # Special case: Wear OS like '4.4W and up'
    match_wear = re.search(r'(\d+\.\d+)W', value)
    if match_wear:
        return match_wear.group(1)

    return None  # if format doesn't match


def is_wear_os(cleaned_value):
    return 'W' in cleaned_value


def is_version_range(cleaned_value):
    return '-' in cleaned_value

In [ ]:
train_df_copy['min_base_os_version'] = train_df_copy['compatible_os_version'].apply(extract_min_base_os)
train_df_copy['is_wear_os'] = train_df_copy['compatible_os_version'].apply(is_wear_os)
train_df_copy['is_version_range'] = train_df_copy['compatible_os_version'].apply(is_version_range)
train_df_copy

* Let's now drop cleaned columns

In [ ]:
train_df_copy.drop(columns=['compatible_os_version', 'release_date'], inplace=True)
train_df_copy

## Split into Categorical & Numerical

In [ ]:
train_df_copy

In [ ]:
def manually_split_columns(df, categorical_list, numerical_list):
    categorical_list = [col for col in categorical_list if col in df.columns]
    numerical_list = [col for col in numerical_list if col in df.columns]

    categorical_df = df[categorical_list].copy()
    numerical_df = df[numerical_list].copy()

    return categorical_df, numerical_df

In [ ]:
categorical_cols = ['app_category', 'reviews_count', 'free_paid', 'age_rating', 'year', 'month', 'day', 'weekday',
                    'quarter', 'is_weekend', 'is_month_start', 'is_month_end', 'week_of_year', 'is_holiday_season',
                    'is_holiday', 'holiday_name', 'min_base_os_version', 'is_wear_os', 'is_version_range']
numerical_cols = ['size_unknown_unit', 'downloads', 'price_if_paid', 'reviews_count_numerical', 'month', 'day',
                  'weekday',
                  'quarter', 'min_base_os_version', 'Y']

cat_df, num_df = manually_split_columns(train_df_copy, categorical_cols, numerical_cols)

In [ ]:
numeric_cols = cat_df.select_dtypes(include=['int32', 'UInt32', 'object']).columns

cat_df[numeric_cols] = cat_df[numeric_cols].astype('category')
cat_df

In [ ]:
cat_cols = num_df.select_dtypes(include=['object']).columns

num_df[cat_cols] = num_df[cat_cols].astype('float64')
num_df

In [ ]:
cat_df['app_name'] = train_df_copy['app_name']
num_df['app_name'] = train_df_copy['app_name']

In [ ]:
num_df.to_csv('cleaned_num_df2.csv', index=False)
cat_df.to_csv('cleaned_cat_df2.csv', index=False)

## Data Analysis

In [ ]:
import pandas as pd
import numpy as np

train_df_copy = pd.read_csv("submissions/cleaned_train_df2.csv")
num_df = pd.read_csv("submissions/cleaned_num_df2.csv")
cat_df = pd.read_csv("submissions/cleaned_cat_df2.csv")

In [ ]:
num_df

In [ ]:
import matplotlib.pyplot as plt


def plot_numeric_distribution(df, column_name, bins=50, color='skyblue'):
    import numpy as np

    plt.figure(figsize=(12, 6))
    plt.hist(df[column_name].dropna(), bins=bins, color=color, edgecolor='black')
    plt.title(f'Distribution of {column_name}', fontsize=14, pad=15)
    plt.xlabel(column_name, fontsize=12)
    plt.ylabel('Frequency', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


### Size
* I have no idea what size is in, but we know it's heavily skewed

In [ ]:
plot_numeric_distribution(num_df, 'size_unknown_unit', 5)

In [ ]:
from scipy.stats import zscore

z = zscore(num_df['size_unknown_unit'])
outliers = num_df[abs(z) > 3]  # Threshold of 3 is commo
outliers

In [ ]:
Q1 = num_df['size_unknown_unit'].quantile(0.25)
Q3 = num_df['size_unknown_unit'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = num_df[(num_df['size_unknown_unit'] < lower_bound) | (num_df['size_unknown_unit'] > upper_bound)]
outliers

In [ ]:
import seaborn as sns

sns.boxplot(x=num_df['size_unknown_unit'])

In [ ]:
from sklearn.ensemble import IsolationForest, RandomForestRegressor

model = IsolationForest(contamination=0.01)
num_df['outlier'] = model.fit_predict(num_df[['size_unknown_unit']])  # -1 = outlier, 1 = inlier
num_df[num_df['outlier'] == -1]

* We will try setting to log
* And try removing them

In [ ]:
num_df['log_size'] = np.log1p(num_df['size_unknown_unit'])
num_df_test = num_df[num_df['outlier'] != -1]

In [ ]:
plot_numeric_distribution(num_df, 'log_size')

* Stick with this for now

In [ ]:
sns.boxplot(x=num_df['log_size'])

### Downloads

In [ ]:
plot_numeric_distribution(num_df, 'downloads')

In [ ]:
sns.boxplot(x=num_df['downloads'])

In [ ]:
num_df['log_downloads'] = np.log1p(num_df['downloads'])
plot_numeric_distribution(num_df, 'log_downloads')
sns.boxplot(x=num_df['log_downloads'])

In [ ]:
from scipy.stats import skew

skew(num_df['downloads'])

In [ ]:
skew(num_df['log_downloads'])

In [ ]:
from scipy.stats import boxcox

num_df['sqrt_downloads'] = np.sqrt(num_df['downloads'])
num_df['downloads_boxcox'], lam = boxcox(num_df['downloads'])
print("Skew after Box-Cox:", skew(num_df['downloads_boxcox']))
print("Skew after sqrt:", skew(num_df['sqrt_downloads']))

In [ ]:
plot_numeric_distribution(num_df, 'downloads_boxcox')
plot_numeric_distribution(num_df, 'sqrt_downloads')

In [ ]:
sns.boxplot(x=num_df.downloads_boxcox)

In [ ]:
sns.boxplot(x=num_df.sqrt_downloads)

### price, reviews count

In [ ]:
plot_numeric_distribution(num_df, 'price_if_paid')
plot_numeric_distribution(num_df, 'reviews_count_numerical')

In [ ]:
skew(num_df['price_if_paid'])

In [ ]:
skew(num_df.reviews_count_numerical)

In [ ]:
num_df['log_price_paid_only'] = num_df['price_if_paid'].where(cat_df['free_paid'] == 'Paid', 0)
num_df['log_price_paid_only'] = np.log1p(num_df['log_price_paid_only'])

In [ ]:
skew(num_df['log_price_paid_only'])

In [ ]:
num_df['log_reviews_count'] = np.log1p(num_df['reviews_count_numerical'])
skew(num_df['log_reviews_count'])

### month, day, weekday, quarter

In [ ]:
print(skew(num_df.month))
print(skew(num_df.day))
print(skew(num_df.weekday))
print(skew(num_df.quarter))

### Os min version

In [ ]:
skew(num_df['min_base_os_version'])

In [ ]:
plot_numeric_distribution(num_df, 'min_base_os_version')

In [ ]:
sns.boxplot(num_df['min_base_os_version'])

In [ ]:
num_df.drop(columns=['outlier', 'app_name'], inplace=True)

In [ ]:
num_df.corr(numeric_only=True)['Y'].sort_values(ascending=False)

In [ ]:
num_df_new = num_df.drop(columns=[
    'app_name', 'size_unknown_unit', 'downloads', 'price_if_paid', 'reviews_count_numerical', 'month', 'day', 'outlier',
    'downloads_boxcox', 'sqrt_downloads',
])

In [ ]:
num_df_new

## Categorical

In [ ]:
cat_df.nunique()


In [ ]:
threshold = 0.01  # 1% of total
for col in cat_df.columns:
    freqs = cat_df[col].value_counts(normalize=True)
    rare = freqs[freqs < threshold]
    if not rare.empty:
        print(f"{col} has rare values:\n{rare}\n")


* We should drop week_of_year
* app_category can have less categories by grouping rare occurences into other

In [ ]:
cat_df.drop(columns=['week_of_year'], inplace=True)

In [ ]:
def check_variation(category_col):
    combined = pd.concat([cat_df[category_col], num_df['Y']], axis=1)
    return combined.groupby(category_col)['Y'].mean().sort_values()


In [ ]:
check_variation("is_month_end")

* We can see is_month_end, is_month_start  has little variation between categories
* So we will drop it

In [ ]:
cat_df.drop(columns=['is_month_end', 'is_month_start'], inplace=True)

In [ ]:
check_variation("is_weekend")

In [ ]:
check_variation("is_holiday")

In [ ]:
check_variation('age_rating')

In [ ]:
check_variation('weekday')

In [ ]:
check_variation('year')

In [ ]:
def group_years(year):
    if year <= 2012:
        return 'Old'
    elif 2013 <= year <= 2016:
        return 'Middle'
    else:
        return 'Recent'


cat_df['year_group'] = cat_df['year'].apply(group_years)
check_variation('year_group')

In [ ]:
check_variation('month')

In [ ]:
check_variation('quarter')

In [ ]:
check_variation('holiday_name')

In [ ]:
def group_holidays(holiday):
    high = [
        "Independence Day", "Veterans Day", "Thanksgiving Day",
        "Memorial Day", "Christmas Day (observed)"
    ]
    mid = [
        "Not Holiday", "Columbus Day", "Martin Luther King Jr. Day",
        "Veterans Day (observed)", "New Year's Day", "Labor Day"
    ]
    low = [
        "Christmas Day", "Washington's Birthday"
    ]

    if holiday in high:
        return "High Rating Holiday"
    elif holiday in mid:
        return "Mid Rating Holiday"
    elif holiday in low:
        return "Low Rating Holiday"
    else:
        return "Other"


cat_df['holiday_group'] = cat_df['holiday_name'].apply(group_holidays)
check_variation('holiday_group')

In [ ]:
check_variation('reviews_count')

In [ ]:
def group_reviews_count(val):
    very_low = ["1+", "5+", "10+", "50+", "100+"]
    low_mid = ["500+", "1,000+", "5,000+", "10,000+", "50,000+"]
    mid = ["100,000+", "500,000+"]
    high = ["1,000,000+", "5,000,000+", "10,000,000+"]
    top = ["50,000,000+", "100,000,000+", "500,000,000+", "1,000,000,000+"]

    if val in very_low:
        return "Very Low"
    elif val in low_mid:
        return "Low-Mid"
    elif val in mid:
        return "Mid"
    elif val in high:
        return "High"
    elif val in top:
        return "Top Tier"
    else:
        return "Other"


cat_df['reviews_group'] = cat_df['reviews_count'].apply(group_reviews_count)
check_variation('reviews_group')

In [ ]:
check_variation('min_base_os_version')

In [ ]:
check_variation('app_category')

In [ ]:
cat_df.nunique()

* drop is weekend is holiday, weekday, month

In [ ]:
cat_df.drop(columns=['is_weekend', 'is_holiday', 'weekday', 'month'], inplace=True)

In [ ]:
cat_df.nunique()

* drop day, min_base_os_version, reviews_count, holiday_name

In [ ]:
cat_df.drop(columns=['day', 'min_base_os_version', 'reviews_count', 'holiday_name'], inplace=True)

In [ ]:
cat_df.nunique()

In [ ]:
check_variation('is_wear_os')

In [ ]:
check_variation('is_version_range')

In [ ]:
check_variation('free_paid')

* drop year, is_holiday_season, app_category_grouped

In [ ]:
cat_df.drop(columns=['year', 'is_holiday_season', 'app_category_grouped'], inplace=True)

In [ ]:
cat_df.nunique()

In [ ]:
num_df_new.nunique()

In [ ]:
num_df_new.drop(columns=['quarter'], inplace=True)

In [ ]:

combined_df = cat_df.join(num_df_new)
combined_df

In [ ]:
combined_df.to_csv('first_combined_df.csv', index=False)

In [ ]:
combined_df.drop(columns=['app_name'], inplace=True)

In [ ]:
combined_df.dtypes

In [ ]:
combined_df

* LinearRegression()
* RandomForestRegressor()
* XGBRegressor()
* GradientBoostingRegressor()

In [ ]:
y = combined_df[['Y']]
X = combined_df.drop(columns=['Y'])

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import OneHotEncoder

cat_pipeline = make_pipeline(
    OneHotEncoder(handle_unknown="ignore"))

preprocessing = make_column_transformer(
    (cat_pipeline, make_column_selector(dtype_include=['object', 'int64'])),
)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessing),
    ('model', RandomForestRegressor())  # Or XGBRegressor(), etc.
])
model_pipeline.fit(X, y.values.ravel())


In [ ]:
from sklearn.model_selection import KFold, cross_val_score

rmses = -cross_val_score(model_pipeline, X, y.values.ravel(),
                         scoring="neg_mean_squared_error", cv=10)

pd.Series(rmses).describe()

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
    'model__max_features': randint(low=2, high=20)}
rnd_search = RandomizedSearchCV(
    model_pipeline, param_distributions=param_distribs, n_iter=10, cv=3,
    scoring='neg_mean_squared_error', random_state=42)
rnd_search.fit(X, y.values.ravel())

In [ ]:
final_model = rnd_search.best_estimator_  # includes preprocessing
feature_importances = final_model["model"].feature_importances_
sorted(zip(feature_importances, final_model["preprocessing"].get_feature_names_out()), reverse=True)

In [ ]:
test_df = pd.read_csv("app-rating-competition/test.csv")

In [ ]:
test_df